<a href="https://colab.research.google.com/github/yamanoyu/atcorder/blob/master/_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Combination

In [ ]:
def cmb(n, r):
    # Combinationを算出 その1
    # 5C2 = 10など
    if n - r < r: r = n - r
    if r == 0: return 1
    if r == 1: return n

    numerator = [n - r + k + 1 for k in range(r)]
    denominator = [k + 1 for k in range(r)]

    for p in range(2, r+1):
        pivot = denominator[p - 1]
        if pivot > 1:
            offset = (n - r) % p
            for k in range(p-1, r, p):
                numerator[k - offset] /= pivot
                denominator[k] /= pivot
    result = 1
    for k in range(r):
        if numerator[k] > 1:
            result *= int(numerator[k])

    return result

In [ ]:
from operator import mul
from functools import reduce

def combination_cnt(n, r):
  # Combinationを算出 その2 (遅い)
  # 5C2 = 10など

  r = min(n - r, r)
  if r == 0: return 1
  return reduce(mul, range(n - r + 1, n + 1)) // reduce(mul, range(1, r + 1))

## 素因数分解

In [ ]:
def prime_factorization(n):
    # 素因数分解 その1
    # prime_factorization(24) の場合 [[2, 3], [3, 1]] (s^3 * 3^1)を返す
    
    arr = []
    tmp = n
    for i in range(2, int((-(-(n ** 0.5)) // 1)) + 1):
        if tmp % i == 0:
            cnt = 0
            while tmp % i == 0:
                cnt += 1
                tmp //= i
            arr.append([i, cnt])

    if tmp != 1:
        arr.append([tmp, 1])

    if arr == []:
        arr.append([n, 1])

    return arr


In [ ]:
def factorization(n):
  # 素因数分解 その2
  # prime_factorization(24) の場合 [[2, 3], [3, 1]] (s^3 * 3^1)を返す
  if n == 1: return []
  factorization_list = []
  i = 2
  _n = n
  while i ** 2 <= n:
    cnt = 0
    while _n % i == 0:
      cnt += 1
      _n //= i
    factorization_list.append([i, cnt])
    i += 1
  
  if _n != 1: factorization_list.append([_n, 1])
  if factorization_list == []: factorization_list.append([n, 1])

  return factorization_list

## フィボナッチ数列

In [ ]:
# これだとすごく遅い
# def fibonacci_sequence(n):
#   if n <= 2:
#     return 1
#   else:
#     return fibonacci_sequence(n - 1) + fibonacci_sequence(n - 2)

# 動的計画法を利用する
def fibonacci_sequence(n):
  fibonacci_sequence_list = [None] * n

  if n == 0:
    return 0
  elif n == 1 or n == 2:
    return 1
  else: 
    fibonacci_sequence_list[0] = 1
    fibonacci_sequence_list[1] = 1
    for i in range(2, n):
      fibonacci_sequence_list[i] = fibonacci_sequence_list[i - 1] + fibonacci_sequence_list[i - 2]

  return fibonacci_sequence_list[n - 1]

fibonacci_sequence(100)

354224848179261915075

## 素数

In [ ]:
# エラトステネスのふるい

def sieve_of_eratosthenes(n):
    # n以下の素数を出力
    prime_num_list = [2]
    number_list = [num for num in range(3, n, 2)]

    while True:
      p = number_list[0]
      if p ** 2 > n:
        return prime_num_list + number_list
        break
      prime_num_list.append(p)
      number_list = [num for num in number_list if num % p != 0]

sieve_of_eratosthenes(100)

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97]

## UnionFind

In [ ]:
class UnionFind:
    # if x is root: self.parents[x] = - (the number of the group nodes)
    # else: self.parents[x] = the parent of x
    def __init__(self, n):
        self.n = n
        self.parents = [-1] * n
 
    # return the parent of x
    def find(self, x):
        history = []
        while self.parents[x] >= 0:
            history.append(x)
            x = self.parents[x]
        for node in history:
            self.parents[node] = x
        return x
 
    # merge the group of x and the group of y
    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)
        if x == y:
            return
        if self.parents[x] > self.parents[y]:
            x, y = y, x
        self.parents[x] += self.parents[y]
        self.parents[y] = x
 
    # return the size of the group of x
    def size(self, x):
        return -self.parents[self.find(x)]
 
    # return whether x and y in a same group
    def same(self, x, y):
        return self.find(x) == self.find(y)
 
    # return [all roots]
    # O(n)
    def roots(self):
        return [i for i, x in enumerate(self.parents) if x < 0]
 
    # return the number of groups
    # O(n)
    def group_count(self):
        return len(self.roots())

## 転倒数

In [ ]:
class Bit:
  # 転倒数
  def __init__(self, n):
    self.size = n
    self.tree = [0] * (n + 1)

  def __iter__(self):
      psum = 0
      for i in range(self.size):
          csum = self.sum(i + 1)
          yield csum - psum
          psum = csum
      raise StopIteration()

  def __str__(self):  # O(nlogn)
      return str(list(self))

  def sum(self, i):
      # [0, i) の要素の総和を返す
      if not (0 <= i <= self.size): raise ValueError("error!")
      s = 0
      while i>0:
          s += self.tree[i]
          i -= i & - i
      return s

  def add(self, i, x):
      if not (0 <= i < self.size): raise ValueError("error!")
      i += 1
      while i <= self.size:
          self.tree[i] += x
          i += i & - i

  def __getitem__(self, key):
      if not (0 <= key < self.size): raise IndexError("error!")
      return self.sum(key+1) - self.sum(key)

  def __setitem__(self, key, value):
      # 足し算と引き算にはaddを使うべき
      if not (0 <= key < self.size): raise IndexError("error!")
      self.add(key, value - self[key])

## Segment Tree

In [ ]:
class SegmentTree:

    __all__ = ['setval', 'pointupdate', 'segquery', 'segsearch_right', 'pointgetval']

    def __init__(self, n=10**6, idetify_elt=-10**9, func=max):
        assert (func(idetify_elt, idetify_elt) == idetify_elt)
        self._n = n
        self._seg_length_half = 2**(n-1).bit_length()
        self._idetify_elt = idetify_elt
        self._seg = [idetify_elt]*(2*self._seg_length_half)
        self._func = func

    def setval(self, x_list):
        '''Set value : A = x_list'''
        assert (len(x_list) == self._n)
        # Set value at the bottom
        for i in range(self._n):
            self._seg[i+self._seg_length_half-1] = x_list[i]    
        # Build value
        for i in range(self._seg_length_half-2, -1, -1):
            self._seg[i] = self._func(self._seg[2*i+1], self._seg[2*i+2])
    
    def pointupdate(self, k, x):
        '''Update : A[k] = x '''
        pos = k + self._seg_length_half - 1
        # Set value at k-th
        self._seg[pos] = x
        # Build bottom-up
        while pos:
            pos = (pos-1)//2
            self._seg[pos] = self._func(self._seg[pos*2+1], self._seg[pos*2+2])
    
    def pointgetval(self, k):
        ''' Return A[k] '''
        return self._seg[k + self._seg_length_half - 1]

    def segquery(self, left, right):
        ''' Return func(A[left], ... , A[right-1]) '''
        # if not left < right
        if right <= left:
            return self._idetify_elt
        
        func_value = self._idetify_elt
        leftpos = left + self._seg_length_half - 1 # leftmost segment
        rightpos = right + self._seg_length_half - 2 # rightmost segment

        while leftpos < rightpos-1:
            if leftpos&1 == 0:
                # if leftpos is right-child
                func_value = self._func(func_value, self._seg[leftpos])
            if rightpos&1 == 1:
                # if rightpos is leftchild
                func_value = self._func(func_value, self._seg[rightpos])
                rightpos -= 1
            # move up
            leftpos = leftpos//2
            rightpos = (rightpos-1)//2
        
        func_value = self._func(func_value, self._seg[leftpos])
        if leftpos != rightpos:
            func_value = self._func(func_value, self._seg[rightpos])
        return func_value

    def segsearch_right(self, condfunc, left=0):
        ''' Return min_i satisfying condfunc( func( A[left], ... , A[i])) 
        if impossible : return n
        '''
        # if impossible (ie. condfunc( func( A[left], ... , A[-1])) is False)
        if not condfunc(self._segquery(left, self._n)):
            return self._n
        
        # possible
        func_value = self._idetify_elt
        rightpos = left + self._seg_length_half - 1
        while True: 
            # while rightpos is the left-child, move bottom-up
            while rightpos&1 == 1:
                rightpos //= 2
            # try
            up_value_trial = self._func(func_value, self._seg[rightpos])
            if not condfunc(up_value_trial):
                # move up and right
                func_value = up_value_trial
                rightpos = (rightpos-1)//2 + 1
            else:
                # move top-down
                while rightpos < self._seg_length_half-1:
                    down_value_trial = self._func(func_value, self._seg[rightpos*2 + 1])
                    if condfunc(down_value_trial):
                        # move left-child
                        rightpos = rightpos*2 + 1
                    else:
                        # move right-child
                        func_value = down_value_trial
                        rightpos = rightpos*2 + 2
                return rightpos - self._seg_length_half + 1

## 深さ優先探索

In [ ]:
import sys


sys.setrecursionlimit(1000000)

tree = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [], [], [], [], [], [], [], []]

def dfs(now):
    print(now, end=' ')
    for i in tree[now]:
        dfs(i)

dfs(1)

1 3 7 8 4 9 10 

## 幅優先探索

In [ ]:
from collections import deque

tree = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [], [], [], [], [], [], [], []]

queue = deque([0])
while queue:
    now = queue.popleft()
    print(now)
    for i in tree[now]:
        queue.append(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


## ナップザック問題

### 重さに注目した動的計画法 [計算量 : $O(NW)$]

In [ ]:
n, max_w = map(int, input().split())
wv = [list(map(int, input().split())) for _ in range(n)]

# i番目までの品物かつ重量の合計j以下で品物を選んだ場合の価値の最大値
dp = [[0] * (max_w + 1) for _ in range(n + 1)]

# 品物i+1を選ばない場合で仮更新してから、ナップサックにi+1個目を詰め込める場合は再更新
for i, (w, v) in enumerate(wv):
    for j in range(max_w + 1):
        dp[i + 1][j] = dp[i][j]
        if j - w >= 0:
            dp[i + 1][j] = max(dp[i + 1][j], dp[i][j - w] + v)

print(dp[-1][-1])

2 10
3 7
5 8
15


### 価値に注目した動的計画法 [計算量 : $O(NV)$]

In [ ]:
import bisect


INF = 10 ** 18

n, max_w = map(int, input().split())
wv = [list(map(int, input().split())) for _ in range(n)]

v_sum = sum([v for w, v in wv])

# 価値iになる時の最小の重さ
dp = [INF] * (v_sum + 1)
dp[0] = 0

for w, v in wv:
    _dp = dp
    for j in range(1, v_sum + 1):
        # j - vが負の場合は0からの遷移としてよい
        idx_prev = max(0, j - v)
        _dp[j] = min(dp[j], dp[idx_prev] + w)
    dp = _dp

# 重さWで達成できる最大価値を求める
print(bisect.bisect_right(dp, max_w) - 1)

2 10
3 5
5 9
[0, 3, 3, 3, 3, 3, 6, 6, 6, 6, 6, 9, 9, 9, 9]
[0, 3, 3, 3, 3, 3, 5, 5, 5, 5, 6, 8, 8, 8, 8]
14


## ダイクストラ法

In [ ]:
import heapq


INF = 10 ** 9
next_node_list = [[(1, 1), (2, 2)], [(2, 2)], []] # (cost, node)


def dijkstra(start, node_num):
    dist = [INF] * node_num
    dist[start] = 0
    visited_list = [False] * node_num

    hq = [(0, start)] # (cost, node)

    while hq:
        visit_node = heapq.heappop(hq)[1] # ノードを pop する
        visited_list[visit_node] = True
        for cost, next_node in next_node_list[visit_node]: # ノード v に隣接しているノードに対して
            if visited_list[next_node] == False and dist[visit_node] + cost < dist[next_node]:
                dist[next_node] = dist[visit_node] + cost
                heapq.heappush(hq, (dist[next_node], next_node))
    return dist

## 強連結成分分解

In [ ]:
# 強連結成分分解(SCC): グラフGに対するSCCを行う
# 入力: <N>: 頂点サイズ, <G>: 順方向の有向グラフ, <RG>: 逆方向の有向グラフ
# 出力: (<ラベル数>, <各頂点のラベル番号>)
def scc(N, G, RG):
    order = []
    used = [0]*N
    group = [None]*N
    def dfs(s):
        used[s] = 1
        for t in G[s]:
            if not used[t]:
                dfs(t)
        order.append(s)
    def rdfs(s, col):
        group[s] = col
        used[s] = 1
        for t in RG[s]:
            if not used[t]:
                rdfs(t, col)
    for i in range(N):
        if not used[i]:
            dfs(i)
    used = [0]*N
    label = 0
    for s in reversed(order):
        if not used[s]:
            rdfs(s, label)
            label += 1
    return label, group

# 縮約後のグラフを構築
def construct(N, G, label, group):
    G0 = [set() for i in range(label)]
    GP = [[] for i in range(label)]
    for v in range(N):
        lbs = group[v]
        for w in G[v]:
            lbt = group[w]
            if lbs == lbt:
                continue
            G0[lbs].add(lbt)
        GP[lbs].append(v)
    return G0, GP
